
#  高性能プログラミングと性能測定(4) --- 命令レベル並列性


# 1. 概要
* 積分の計算を題材にSIMDを用いて高速化する手法を学んだ
* SIMD化自体はうまくいったが, まだプロセッサ(1コア)の最高性能には遠い
* さらに性能をあげるにはプロセッサが複数命令を並行して実行できる仕組み(命令レベル並列性; スーパースカラー並列)をうまく発動させる必要がある



# 2. 復習 単純なSIMD化を施した積分の性能
* 前回, 以下のSIMD化されたコードが, ループ1周を(Skylakeで) 4 processor cyclesで実行できることを見た
* その理由はvfmaddpd 命令間の依存関係と遅延にあった


In [ ]:
%%writefile int_x2_simd.c
#include <stdio.h>
#include <stdlib.h>

/* doubleを8つ並べたデータ型(doublev) */
typedef double doublev __attribute__((vector_size(64),aligned(sizeof(double))));
enum { n_lanes = sizeof(doublev) / sizeof(double) };

double int_x2_simd(double a, double b, long n) {
  doublev s = {0,0,0,0,0,0,0,0};
  // n をレーン数の倍数に
  n += n_lanes - 1;
  n -= n % n_lanes;
  double dx = (b - a) / (double)n;
  doublev x = {a,a+dx,a+2*dx,a+3*dx,a+4*dx,a+5*dx,a+6*dx,a+7*dx};
  for (long i = 0; i < n; i += n_lanes) {
    s += x * x;
    x += n_lanes * dx;
  }
  return (s[0] + s[1] + s[2] + s[3] + s[4] + s[5] + s[6] + s[7]) * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_x2_simd(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
gcc -O3 -march=native int_x2_simd.c -o int_x2_simd

In [ ]:
./int_x2_simd


# 3. プロセッサの限界性能
* 一方でOakbridge CXのプロセッサ(Skylake, Cascade Lake; にかぎらず最近のIntelのCPU)の最高性能は, SIMD化されたfmadd命令を「1サイクルに2個」実行できるというもので, つまり今得られている性能はその性能からは8倍の開きがある(2/1 : 1/4 = 8 : 1)
* 1つの命令の実行に4サイクルかかるにもかかわらず, 1サイクルに2個実行できるとはどういうことなのか? 
* それは, <font color="blue">依存関係のない命令たち</font>は, 前の命令の結果が出るのを待たずして実行を始められる(つまり, 前の命令とオーバーラップして実行できる), という, プロセッサの機能による(<font color="blue">命令レベル並列性, スーパースカラ実行</font>)
* SIMD化されたコードで1つのvfmaddpd命令に4サイクルかかっていた理由はそれらが依存関係を持っていたからで, 依存関係を持っていた理由は, 和を計算するために同じ変数sに x * x を足していたからであった(前の足し算が終わらないと次の足し算を始められない).
```
for (...) {
  s += x * x;
  ...
}
```

* 命令の遅延自身を(ソフトウェアで工夫をして)縮めることは出来ない
* これ以上高速化するには, 和を計算するのに複数の変数を使えば良い(使うしかない)
* たとえば 2個の変数 s0, s1を用いて, 半分は s0 に, もう半分は s1 に足していく
* やっている計算の量は一切変わらないのだが, s0 を使った計算とs1を使った計算の間には依存関係がなくなるので, 並行して実行できる. つまり, 命令レベル並列性が向上する

* それを愚直に行ったものが以下


In [ ]:
%%writefile int_x2_simd2.c
#include <stdio.h>
#include <stdlib.h>
typedef double doublev __attribute__((vector_size(64),aligned(sizeof(double))));

double int_x2_simd_ilp(double a, double b, long n) {
  doublev s0 = { 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0 };
  doublev s1 = { 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0 };
  double dx = (b - a) / (double)n;
  doublev x0 = { a,      a+  dx, a+ 2*dx, a+ 3*dx, a+ 4*dx, a+ 5*dx, a+ 6*dx, a+ 7*dx };
  doublev x1 = { a+8*dx, a+9*dx, a+10*dx, a+11*dx, a+12*dx, a+13*dx, a+14*dx, a+15*dx };
  for (long i = 0; i < n; i += 16) {
    s0 += x0 * x0;
    s1 += x1 * x1;
    x0 +=  16 * dx;
    x1 +=  16 * dx;
  }
  doublev s = s0 + s1;
  return (s[0] + s[1] + s[2] + s[3] + s[4] + s[5] + s[6] + s[7]) * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_x2_simd_ilp(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
gcc -O3 -march=native int_x2_simd2.c -o int_x2_simd2

In [ ]:
./int_x2_simd2


* SIMD化しただけのものと, 変数を二つ用いて命令レベル並列性を増加させたものの性能を比べる


In [ ]:
time ./int_x2_simd

In [ ]:
time ./int_x2_simd2


* ほぼ2倍になっている

* ということはこの調子で変数を増やしていけばさらに性能が向上すると見込まれる
* 変数が1つ増える度にいちいちプログラムを大幅に書き換えなくて済むようにしたものが以下
* enum { nc = ... } の定義を書き換えるだけで変数の数を調節できる


In [ ]:
%%writefile int_x2_simd_nc.c
#include <stdio.h>
#include <stdlib.h>

typedef double real;
typedef real realv __attribute__((vector_size(64),aligned(sizeof(real))));

enum { nl = sizeof(realv) / sizeof(real) };
enum { nc = 4 };

real int_x2_simd_nc(real a, real b, long n) {
  realv s[nc];
  realv x[nc];
  real dx = (b - a) / (real)n;
  for (long i = 0; i < nc; i++) {
    for (long j = 0; j < nl; j++) {
      s[i][j] = 0.0;
    }
  }
  for (long i = 0; i < nc; i++) {
    for (long j = 0; j < nl; j++) {
      x[i][j] = a + (nl * i + j) * dx;
    }
  }
  asm volatile("# ============= int_x2_simd_nc loop");
  for (long i = 0; i < n; i += nc * nl) {
    for (long j = 0; j < nc; j++) {
      s[j] += x[j] * x[j];
      x[j] += nc * nl * dx;
    }
  }
  asm volatile("# ------------- int_x2_simd_nc loop");
  realv ss = s[0];
  for (long j = 1; j < nc; j++) {
    ss += s[j];
  }
  real sss = ss[0];
  for (long j = 1; j < nl; j++) {
    sss += ss[j];
  }
  return sss * dx;
}

int main(int argc, char ** argv) {
  real a = (argc > 1 ? atof(argv[1]) : 0.0);
  real b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  real s = int_x2_simd_nc(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
gcc -O3 -march=native int_x2_simd_nc.c -o int_x2_simd_nc

In [ ]:
time ./int_x2_simd_nc


* nc=4 で生成されたアセンブリを覗いてみる

In [ ]:
gcc -O3 -S -march=native int_x2_simd_nc.c
cat int_x2_simd_nc.s


* 関係すると思われるところの抜粋
* 一回の繰り返しの中で s += x * x から出てきたと思われる vfmadd 命令4つと, x += 32 * dx から出てきたと思われる vaddpd 命令4つが並んでいる. 
* 注目は vfmadd 命令で使われているのが別のレジスタである, つまり, 依存関係がないということ. vaddpd 間にも依存関係ない. したがって4つのvfmadd, 4つのvaddpdは並行して実行できる
```
.L3:
        addq    $32, %rax
        vfmadd231pd     %zmm5, %zmm5, %zmm9
        vfmadd231pd     %zmm4, %zmm4, %zmm8
        cmpq    %rdx, %rax
        vfmadd231pd     %zmm3, %zmm3, %zmm7
        vfmadd231pd     %zmm2, %zmm2, %zmm6
        vaddpd  %zmm5, %zmm1, %zmm5
        vaddpd  %zmm4, %zmm1, %zmm4
        vaddpd  %zmm1, %zmm3, %zmm3
        vaddpd  %zmm1, %zmm2, %zmm2
        jne     .L3
```



* C++であればテンプレートを使って, 同じプログラム中に複数のncを同居させることも簡単にできる

In [ ]:
%%writefile int_x2_simd_template.cc
#include <stdio.h>
#include <stdlib.h>

typedef double real;
typedef real realv __attribute__((vector_size(32),aligned(sizeof(real))));

enum { nl = sizeof(realv) / sizeof(real) };

template<int nc>
real int_x2_simd_template(real a, real b, long n) {
  realv s[nc];
  realv x[nc];
  real dx = (b - a) / (real)n;
  real x_ = a;
  for (long i = 0; i < nc; i++) {
    for (long j = 0; j < nl; j++) {
      s[i][j] = 0.0;
    }
  }
  for (long i = 0; i < nc; i++) {
    for (long j = 0; j < nl; j++) {
      x[i][j] = a + (nl * i + j) * dx;
    }
  }
  asm volatile("# ============= int_x2_simd_template loop");
  for (long i = 0; i < n; i += nc * nl) {
    for (long j = 0; j < nc; j++) {
      s[j] += x[j] * x[j];
      x[j] += nc * nl * dx;
    }
  }
  asm volatile("# ------------- int_x2_simd_template loop");
  realv ss = s[0];
  for (long j = 1; j < nc; j++) {
    ss += s[j];
  }
  real sss = ss[0];
  for (long j = 1; j < nl; j++) {
    sss += ss[j];
  }
  return sss * dx;
}

int main(int argc, char ** argv) {
  real a = (argc > 1 ? atof(argv[1]) : 0.0);
  real b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  real s = int_x2_simd_template<4>(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
g++ -O3 -march=native int_x2_simd_template.cc -o int_x2_simd_template


* この性能がどのくらい限界に近いかを考察する
* 復習: 限界性能 1 processor cycleにつき, SIMD化された浮動小数点命令(fmadd, mul, addなど)が2つ
 * ここで初めて明かした事実だが, addもmulもfmaddもどれも同じ1と数えられる
 * 1 processor cycleにつき, vfmadd x 2, vfmadd x 1 + mul x 1, vfmadd x 1 + add x 1, mul x 1 + add x 1 などどんな組み合わせも可能だが, vfmadd x 1 + mul x 1 + add x 1 とかはできない(演算量としては vfmadd 2 個分には違いないのだが)

<font color="blue">on Oakbridge CX</font> 

In [ ]:
perf stat ./int_x2_simd_template

* 以下は(例によってこのページ上では失敗するので) Oakbridge CXでの結果

```
01jupyter$ perf stat ./int_x2_simd_nc
 Performance counter stats for './int_x2_simd_nc':

             65.94 msec task-clock                #    0.979 CPUs utilized
                 5      context-switches          #    0.076 K/sec
                 1      cpu-migrations            #    0.015 K/sec
               210      page-faults               #    0.003 M/sec
       164,007,164      cycles                    #    2.487 GHz
       350,799,322      instructions              #    2.14  insn per cycle
	32,562,169      branches                  #  493.838 M/sec
            33,473      branch-misses             #    0.10% of all branches

       0.067373817 seconds time elapsed

       0.064476000 seconds user
       0.002014000 seconds sys
```

* `164,007,164      cycles` をもとに, 1 cycle で何flops 実行されいてるかを計算する
* 復習: 限界性能は, 

$$ 8 \mbox{ (SIMD変数1つに収まるdouble型の数)} \times 2 \mbox{ (fmaddで2 flops)} \times 2 \mbox{ (1 cycleに実行可能なfmadd命令数)} = 32 \mbox{ flops/cycle} $$

* この計算全体では,
 * s += x * x  1回につき 2 flops
 * dx += ... 1回につき 1 flop
だから 3 * 1000000000 flops 費やしている

* よって1 cycleあたりのflops数は

$$ 3 * 1000000000 / 164007164 = 18.2918... $$

* 限界が32で, 得られているのが$\approx$ 10. だいたい限界性能の57.1%

* ところでこの計算においては原理的に75%までしか得られないことに注意. というのも fmadd と add を同数だけ実行する必要があり, プロセッサの限界は, fmadd, add, mul を(どれも1と数えて) 2 個/cycle なので.

* それを考慮した(この計算における)限界 flops/cycle は 

$$ 16 + 8 = 24 \mbox{flops/cycle} $$

それと比較すると, 76%程度出ている

* 同じことだが別の角度から, 「ループ一周に何サイクルかかっているか」を計算するのも有用である

* 上記のループ `.L3: ... jg .LG` は一周につき vfmadd を5回実行しているため, 1000000000 の点に対して被積分関数(x * x)を計算するには, 

$$ 1000000000 / (5 * 8) = 25000000 $$

周, 回っているはずである

* それが 164,007,164 サイクルで実行できているというのだから, 1周回るのに,

$$ 164007164 / 50000000 = 6.56 ... $$

サイクルかかっている計算になる

* 我々が望むことはこのループが1周につき(vfmaddの遅延である)4 cycleで回るということで, それがどうやら今は6.56 cycleとなっているらしい

* 故に, 限界性能に対する性能は 4/6.56 =61%

* 別の味方として, このループの実行に, どうやら期待よりも余分に2.5 cycleがかかっているようだということ. その原因はこれまで述べたことだけからは説明できないが, 少なくともこれ以上改善しようと思ったときのよい足がかりになる

* なお, 上記で行った計算をする代わりにperfを使って値を見てしまうのも良い方法

```
t67005@obcx03:01jupyter$ perf stat -e cycles -e branches -e fp_arith_inst_retired.512b_packed_double ./int_x2_simd_nc  
s = 0.333333

 Performance counter stats for './int_x2_simd_nc':

        159,751,756      cycles
         31,997,158      branches
        375,000,003      fp_arith_inst_retired.512b_packed_double
        
       0.049367224 seconds time elapsed
       0.047036000 seconds user
       0.002001000 seconds sys
```

* branches がほぼループの繰り返し回数だと思って間違いない
* ループ一周のサイクル数 = cycles / branches がほぼ5という事実が直ちに得られる
* これとループ中の命令数 4 fmadd + 4 add から, 理想的な実行スループットは (4 + 4) / 2 = 4 cycles/周
* よって理想的なcycle数に加え, 1 cycle 余分にかかっている
